# Step 4
Se descargan todas las dependencias necesarias para leer los archivos CSV, XLS y hacer la conexión con la base de datos en MySQL

In [ ]:
!pip install pymysql
!pip install pandas
!pip install openpyxl
!pip install numpy

In [ ]:
import csv
import pymysql
import pandas as pd

#se abre el archivo csv de las estaciones
with open('../data/stations.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    #se inicia la conexion con MySQL
    conn = pymysql.connect(host="localhost", port=3306, user="root", passwd="", db="weather")
    cur = conn.cursor()
    #conforme se va leyendo las lineas del archivo, se van insertando en la base de datos
    for row in reader:
        sql = "INSERT INTO `stations` (`station_id`, `name`,`latitude`,`longitude`,`elevation`) VALUES (%s, %s,%s,%s,%s)"
        try:
            cur.execute(sql, (row['station_id'],row['name'],row['latitude'],row['longitude'],row['elevation']))
            conn.commit()
            print("registro insertado")
        except:
            conn.rollback()
            print("no hubo registro")
    cur.close()
    conn.close()


In [ ]:
#se abre el archivo csv de las variables
with open('../data/variables.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    #se inicia la conexion con MySQL
    conn = pymysql.connect(host="localhost", port=3306, user="root", passwd="", db="weather")
    cur = conn.cursor()
    #conforme se va leyendo las lineas del archivo, se van insertando en la base de datos
    for row in reader:
        sql = "INSERT INTO `variables` (`variable_id`, `name`, `description`, `units`) VALUES (%s, %s,%s,%s)"
        try:
            cur.execute(sql, (row['variable_id'],row['name'],row['description'],row['units']))
            conn.commit()
            print("registro insertado")
        except:
            conn.rollback()
            print("no hubo registro")
    cur.close()
    conn.close()

Se importa la librería para conexión con base de datos, luego se llama a los archivos csv para leer e insertar los registros en la base de datos.

In [ ]:
import pandas as pd
#se definen las direcciones de los archivos
station1 =["../step3/station1/2019.xlsx","../step3/station1/2020.xlsx","../step3/station1/2021.xlsx"]
station2 =["../step3/station2/2019.xlsx","../step3/station2/2020.xlsx","../step3/station2/2021.xlsx"]
station3 =["../step3/station3/2019.xlsx","../step3/station3/2020.xlsx","../step3/station3/2021.xlsx"]
station4 =["../step3/station4/2019.xlsx","../step3/station4/2020.xlsx","../step3/station4/2021.xlsx"]
station5 =["../step3/station5/2019.xlsx","../step3/station5/2020.xlsx","../step3/station5/2021.xlsx"]

def insertar_registros(station,paths):
    tam = len(paths)
    if tam > 0:
        for path in paths:       
            #se define el header a partir de la segunda fila, debido a que en la primera está el titulo del documento
            df = pd.read_excel(path,header=1)
            #data = df.iloc[1::1]
            column_date = df.columns[0]
            #column_grades=df.columns[1]
            print(column_date)
            #print(column_grades)
            print("-" * len(column_date))
            for index, row in df.iterrows():
                print(row[column_date])
            #print(data)
            #print(df)
    else:
        print("tu arreglo está vacio")

insertar_registros(1,station1)
#insertar_registros(2,station2)
#insertar_registros(3,station3)
#insertar_registros(4,station4)
#insertar_registros(5,station5)